In [69]:
#importer les librairies
import numpy as np
import  pandas as pd


In [70]:
Data=pd.read_csv('/kaggle/input/jobs-in-data/jobs_in_data.csv')
Data.duplicated().sum()
Data.drop_duplicates(inplace=True)
Data.drop(['salary','work_year'],axis=1, inplace=True)


In [71]:
Data


,job_title,job_category,salary_currency,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,Data DevOps Engineer,Data Engineering,EUR,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,Data Architect,Data Architecture and Modeling,USD,186000,United States,Senior,Full-time,In-person,United States,M
2,Data Architect,Data Architecture and Modeling,USD,81800,United States,Senior,Full-time,In-person,United States,M
3,Data Scientist,Data Science and Research,USD,212000,United States,Senior,Full-time,In-person,United States,M
4,Data Scientist,Data Science and Research,USD,93300,United States,Senior,Full-time,In-person,United States,M
...,...,...,...,...,...,...,...,...,...,...
9350,Data Specialist,Data Management and Strategy,USD,165000,United States,Senior,Full-time,Remote,United States,L
9351,Data Scientist,Data Science and Research,USD,412000,United States,Senior,Full-time,Remote,United States,L
9352,Principal Data Scientist,Data Science and Research,USD,151000,United States,Mid-level,Full-time,Remote,United States,L
9353,Data Scientist,Data Science and Research,USD,105000,United States,Entry-level,Full-time,Remote,United States,S


In [72]:
data = Data.copy()


In [73]:

# Count the occurrences of each job title
job_title_counts = data['job_title'].value_counts()

# Identify job titles with 3 or fewer occurrences
job_title_cat_other = job_title_counts[job_title_counts <= 3].index

# Replace rare job titles with 'other'
data['job_title'] = data['job_title'].apply(lambda x: 'other' if x in job_title_cat_other else x)


# Count the occurrences of each employee residence
emp_res_counts = data['employee_residence'].value_counts()

# Identify employee residences with 2 or fewer occurrences
emp_res_cat_other = emp_res_counts[emp_res_counts <= 2].index

# Replace rare employee residences with 'other'
data['employee_residence'] = data['employee_residence'].apply(lambda x: 'other' if x in emp_res_cat_other else x)


# Count the occurrences of each company location
com_loc_counts = data['company_location'].value_counts()

# Identify company locations with 2 or fewer occurrences
com_loc_cat_other = com_loc_counts[com_loc_counts <= 2].index

# Replace rare company locations with 'other'
data['company_location'] = data['company_location'].apply(lambda x: 'other' if x in com_loc_cat_other else x)


In [74]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

train_data, test_data = train_test_split(data,test_size=0.2,random_state=38)
train = data.drop('salary_in_usd',axis=1)
train_labels = data['salary_in_usd'].copy()


pipeline = Pipeline([
    ('one_hot_cat',OneHotEncoder())
])
 

train_pipelined = pipeline.fit_transform(train)
train_pipelined.shape
test = test_data.drop('salary_in_usd',axis=1)
test_labels = test_data['salary_in_usd'].copy()
test_pipelined = pipeline.transform(test)


In [75]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Créer un modèle RandomForestRegressor
rf_model = RandomForestRegressor()

# Ajuster le modèle aux données d'entraînement
rf_model.fit(train_pipelined, train_labels)

# Prédire sur les données de test
rf_pred = rf_model.predict(test_pipelined)

# Calculer les métriques d'évaluation
rf_r2 = r2_score(test_labels, rf_pred)
rf_mse = mean_squared_error(test_labels, rf_pred)
rf_mae = mean_absolute_error(test_labels, rf_pred)

print(f"Score R² pour RandomForestRegressor : {rf_r2}")
print(f"Mean Squared Error pour RandomForestRegressor : {rf_mse}")
print(f"Mean Absolute Error pour RandomForestRegressor : {rf_mae}")

Score R² pour RandomForestRegressor : 0.52100222876883
Mean Squared Error pour RandomForestRegressor : 2166183084.427537
Mean Absolute Error pour RandomForestRegressor : 34589.340154959194


In [76]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# Create DecisionTreeRegressor model
dt_model = DecisionTreeRegressor()

# Fit the model to the training data
dt_model.fit(train_pipelined, train_labels)

# Make predictions on the test data
dt_pred = dt_model.predict(test_pipelined)

# Calculer les métriques d'évaluation
dt_r2 = r2_score(test_labels, dt_pred)
dt_mse = mean_squared_error(test_labels, dt_pred)
dt_mae = mean_absolute_error(test_labels, dt_pred)

print(f"Score R² pour DecisionTreeRegressor : {dt_r2}")
print(f"Mean Squared Error pour DecisionTreeRegressor : {dt_mse}")
print(f"Mean Absolute Error pour DecisionTreeRegressor : {dt_mae}")

Score R² pour DecisionTreeRegressor : 0.5317503404674067
Mean Squared Error pour DecisionTreeRegressor : 2117576641.664449
Mean Absolute Error pour DecisionTreeRegressor : 33302.235434505135


In [77]:
from joblib import dump

# Sauvegarder le modèle
dump(dt_model, 'modele_regression.pkl')

['modele_regression.pkl']